# задача
создание нейро-финансиста в торговой компании.

В его обязанности входит:
1. Заполнение журнала операции.
2. Уточнение по непольным данным.
3. Подготовка и написание аналитических записок.

# подготовка окружения

In [2]:
# # загружаем модули
# !pip install -q langchain-community langchain-openai faiss-cpu

In [3]:
# импортируем библиотеки
import requests
import os
import re
import time
import pandas as pd
import ast

from dotenv import load_dotenv
# from google.colab import userdata
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

# # активируем ключ api от open ai
# os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key_lessons')
load_dotenv('.env')

True

# база данных

## db

In [4]:
# функция для загрузки базы данных
def load_document_text(url: str) -> str:
    #
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('invalid google docs url')
    doc_id = match_.group(1)

    #
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
# выводим загруженный текст db_v4
database = load_document_text('https://docs.google.com/document/d/1l76Ly7zW4ve4hRBrSuGJaJeLwqk2G8N9OFQYKtcPDtU/edit?usp=sharing')
print(database[:1000])

0


In [6]:
# преобразовывем текст базы данных в формат markdown
def text_to_markdown(text: str) -> str:
    def second_level(text):
        # ищем загаловки второго уровня и заменяем значения
        return re.sub(
            r'^(\d+\.\d+)\.\s+(.+)',
            r'## \1. \2\n\1. \2',
            text,
            flags=re.MULTILINE
        )

    def first_level(text):
        # ищем загаловки первого уровня и заменяем значения
        return re.sub(
            r'^(\d+)\.\s+(.+)',
            r'# \1. \2',
            text,
            flags=re.MULTILINE
        )

    # Порядок важен: сначала вложенные, потом основные
    text = second_level(text)
    text = first_level(text)
    return text


In [7]:
markdown = text_to_markdown(database)
print(markdown[:1000])

0


In [8]:
# фунция разделитель для markdown разметки
def split_text(text: str) -> str:
    headers_to_split = [
        ('#', 'Header 1'),
        ('##', 'Header 2')
    ]

    # определяем сплиттер
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split)
    database_md = markdown_splitter.split_text(text)

    # возвращаем получнное значение
    return database_md


In [9]:
split_text(markdown[:1000])

[Document(metadata={}, page_content='0')]

In [10]:
# создаем переменную список чанков
source_chunks = []

# добавляем отрывки документов в список
source_chunks = split_text(markdown)
source_chunks[:3]

[Document(metadata={}, page_content='0')]

In [11]:
# убираем пустые чанки и выводим итоговое кол-во
len(source_chunks)

1

In [12]:
# определяем эмбеддинги, помещаем их в векторное хранилище и инициализируем клиента
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(source_chunks, embeddings)

client = OpenAI()

In [13]:
# Определяем путь для сохранения в локальном хранилище
folder_path = 'content/'

# Имя файла для сохранения
index_name = 'db'

# Сохраняем db_from_texts в локальное хранилище
db.save_local(folder_path=folder_path, index_name=index_name)

## df

In [14]:
#
path_download = 'https://drive.google.com/uc?export=download&id='

In [15]:
# загружаем необходимые таблицы
# таблица массив для тестирования accounting
path_download_ok = path_download + '1dBLApOXHmr_JlD_6N2RXGcVlSLEQjP07'
df_ok = pd.read_csv(path_download_ok)
df_ok

,note,date of oper,sum,account,counterparty,category,date of note,id
0,12.03.24 в магазине на Ленина сегодня выручка ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"25 ноября в магазине ""Экспресс"" собрали 52 тыс...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"5 июня 2025 выручка в ""Модный дом"" — около 104...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"18 июля 2025 с автоматов в ""Сити-Плазе"" собрал...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"3 февраля 2025 с автомата в ""Городке"" пришло п...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,12 мая 2024 с *2487 перекинул 50 000 на карту...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,3 ноября 2023 ушло 80 000 с *2313 на счёт *2487.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,7 июня 2025 перевёл 100 000 со *1231 на расчёт...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,12.03.24 в магазине на Ленина сегодня выручка ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# таблица массив для тестирования ask
path_download_ask = path_download + '1dIwEUvlxJ6EEuw8z8SjhYGQh5MDBCeJZ'
df_ask = pd.read_csv(path_download_ask)
df_ask

,note,date of oper,sum,account,counterparty,category,date of note,id
0,"Всем привет! Закрыли смену, выручка за сегодня...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Только что продали классную кожаную куртку за ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ребят, распродажа просто взлетела! Продали 50 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Привет, команда! Автомат на станции ""Восточная...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Ура! Автомат в бизнес-центре ""Солнечный"" побил...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,Зачислен купонный доход по ранее приобретенным...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Перевела 200 000 руб. с основного расчетного с...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Осуществила перевод 150 000 руб. с основного с...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Перечислила 50 000 руб. с текущего счета на на...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# таблица массив для тестирования analyze
path_download_analyze = path_download + '13deuMBOzcP1y0EkOPVhv6SG3crd-y9_E'
df_analyze = pd.read_csv(path_download_analyze)
df_analyze

,note,date of oper,sum,account,counterparty,category,date of note,id
0,"Пришли, пожалуйста, аналитическую записку по д...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Подготовь, пожалуйста, записку с анализом дене...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Жду от тебя краткую записку по движению средств.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Сделай, пожалуйста, отчет по поступлениям и ра...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Нужна аналитика движения денежных средств — сд...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,Жду аналитическую записку по движению средств.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Прошу прислать краткий отчет по финансовым пот...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,"Пришли, пожалуйста, аналитическую записку по д...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,"Подготовь, пожалуйста, записку с анализом дене...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# таблица для analyze
path_download_sheet = path_download + '1188mxc6_X-NOfGeDqlhbgqUk-O3U_tKn'
df_sheet = pd.read_csv(path_download_sheet)
df_sheet

,Остаток на начало месяца,"500 000,00"
0,NaN,NaN
1,1. Денежные потоки от операционной деятельности,"(605 000,00)"
2,NaN,NaN
3,1.1. Продажи через торговые точки,"1 200 000,00"
4,1.2. Продажи через вендинговые автоматы,"350 000,00"
5,1.3. Возвраты от поставщиков,"80 000,00"
6,1.4. Закупка товара,"(900 000,00)"
7,1.5. Транспортные услуги,"(120 000,00)"
8,1.6. Комиссии за эквайринг,"(45 000,00)"
9,1.7. Расчетно-кассовое обслуживание (РКО),"(15 000,00)"


In [19]:
# таблица для заполнения и итогового тестирование
path_download_total = path_download + '1sUaJDIGhrJSGFSlgerPwwBn9vY6lLSNd'
df_total = pd.read_csv(path_download_total)
df_total

,note,date of oper,sum,account,counterparty,category,date of note,id


In [20]:
# функция для заполнения таблицы полученными значениями нейро-сотрудником
def note_to_sheet(lst, note, df):
    # индексы для ввода данных
    index_row = len(df)

    # записываем значения в таблицу
    df.loc[index_row] = {
        'note':note,
        'date of oper':lst[0],
        'sum':lst[1],
        'account':lst[2],
        'counterparty':lst[3],
        'category':lst[4]
    }

    # сохранение значении в документ
    df.to_csv('test.csv', index=False)

# агенты

## accounting

In [21]:
# прописываем роль модели для заполнения таблицы, версию и температуру
system_for_accounting = '''
Ты — великолепный сотрудник финансового отдела торговой компании.
У тебя отлично получается извлекать и классифицировать важные сущности из кратких
сообщений о движении денежных средств. \n\n

Пожалуйста, извлеки только 5 важных сущности из сообщения.
Ознакомся с кратим содержанием, чтобы быть в контексте.
Используй предоставленную документацию для определения статьи.
Строго придерживайся формата данных - список:
[дата, сумма, счёт, контрагент, статья] \n\n

Ты знаешь что:
1. Денежные потоки от операционной деятельности
1.1. Продажи через торговые точки
Описание: Выручка от продажи товаров через физические магазины и киоски. 
Мы продаем и реализуем товары физическим лицам или покупателям, которые приходят 
в наши торговый точки.
Группа: Поступление.
Включает: Наличные платежи клиентов; Безналичные платежи через терминалы эквайринга; 
Оплата банковскими картами через онлайн платёжные системы.

1.2. Продажи через вендинговые автоматы
Описание: Выручка от автоматизированных систем продаж (вендинговых аппаратов). 
Физические лица или покупатели пользуются нашими автоматами по продажам.
Группа: Поступление.
Включает: Наличные деньги из касс автоматов; Безналичные платежи через электронные 
системы оплаты (QR-коды, NFC).

1.3. Возвраты от поставщиков
Описание: Суммы, полученные от поставщиков за возвращенный товар. Полученные суммы 
от наших поставщиков за некачественный товар. 
Группа: Поступление.
Включает: Компенсацию за бракованный товар; Возврат предоплаты за непоставленный 
товар; Реституцию по договору поставки.

1.4. Закупка товара
Описание: Оплата товаров для перепродажи. Покупка товаров у наших партнеров и 
поставщиков для торговли через торговые точки или автоматы.
Группа: Выбытие.
Включает: Оплата основных партий товара; Предоплаты поставщикам; Погашение кредитов 
перед поставщиками.

1.5. Транспортные услуги
Описание: Расходы на доставку товаров и документов. Все уплаты связанные с 
логистикой товаров от наших партнеров и поставщиков. 
Группа: Выбытие.
Включает: Услуги транспортных компаний; Аренда грузового транспорта; ГСМ для 
собственного автопарка; Курьерские услуги.

1.6. Комиссии за эквайринг
Описание: Проценты, удерживаемые банками за прием безналичных платежей. 
Группа: Выбытие.
Включает: Комиссии за использование POS-терминалов; Комиссии за онлайн-эквайринг.

1.7. Расчётно-кассовое обслуживание (РКО)
Описание: Расходы на обслуживание расчетного счета компании.
Группа: Выбытие.
Включает: Ежемесячная абонентская плата за РКО; Комиссии за переводы и платёжные 
поручения; Открытие и закрытие счетов.

1.8. Налоги (ЕНВД, УСН 6%)
Описание: Обязательные платежи в бюджет государства.
Группа: Выбытие.
Включает: Уплату налогов согласно выбранному режиму налогообложения; Авансовые 
платежи.

1.9. Зарплаты и налоги (ФОТ) производственного персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для 
производственного, рабочего персонала. Налоги при выплате зарплат.
Группа: Выбытие.
Включает: Заработную плату рабочих, операторов, складских сотрудников; НДФЛ с 
зарплаты производственного персонала; Взносы в ПФР, ФОМС, ФСС за производственный 
персонал.

1.10. Зарплаты и налоги (ФОТ) коммерческого персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для коммерческого 
персонала. Налоги при выплате зарплат.
Группа: Выбытие.
Включает: Заработную плату менеджеров по продажам, маркетологов, торговых 
представителей; НДФЛ с зарплаты коммерческого персонала; Взносы в ПФР, ФОМС, ФСС за 
коммерческий персонал.

1.11. Зарплаты и налоги (ФОТ) административного персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для 
административного персонала. Налоги при выплате зарплат.
Группа: Выбытие.
Включает: Заработную плату руководителей, бухгалтеров, юристов, IT-специалистов, 
HR-менеджеров; НДФЛ с зарплаты административного персонала; Взносы в ПФР, ФОМС, ФСС 
за административный персонал.

1.12. Обучение персонала
Описание: Инвестиции в профессиональное развитие сотрудников.
Группа: Выбытие.
Включает: Оплату курсов, тренингов, семинаров; Подписку на образовательные платформы; 
Материалы для обучения.

1.13. Расходы на персонал
Описание: Социальные и мотивационные затраты на сотрудников. Корпоративы и годовщины 
компании. 
Группа: Выбытие.
Включает: Подарки сотрудницам и сотрудникам; Корпоративные мероприятия; Организацию 
праздников; Аренда залов, кафе, ресторанов для мероприятии. 

1.14. Командировочные расходы
Описание: Расходы, связанные с деловыми поездками сотрудников.
Группа: Выбытие.
Включает: Проездные билеты; Проживание в отелях; Суточные.

1.15. Представительские расходы
Описание: Расходы на поддержание деловых отношений.
Группа: Выбытие.
Включает: Бизнес-ланчи с партнёрами; Подарки клиентам и контрагентам; Организацию 
мероприятий для клиентов; Аренда кафе и ресторанов.

1.16. Поиск и найм персонала
Описание: Расходы на привлечение новых сотрудников.
Группа: Выбытие.
Включает: Оплату услуг рекрутинговых агентств; Подписку на платформы 
(hh.ru, SuperJob); Публикацию вакансий.

1.17. Реклама и маркетинг
Описание: Расходы на продвижение бизнеса.
Группа: Выбытие.
Включает: Контекстную рекламу (Яндекс.Директ, Google Ads); Баннерную рекламу; Услуги 
дизайнеров, копирайтеров; Работу с PR-агентствами.

1.18. Содержание торговых точек и офиса
Описание: Расходы на поддержание рабочих помещений.
Группа: Выбытие.
Включает: Покупка канцтоваров; Закупка хозяйственных товаров; Уборку; Мелкий ремонт; 
Покупка оборудования до 10 000.

1.19. Аренда
Описание: Платежи за использование недвижимости.
Группа: Выбытие.
Включает: Арендную плату за торговые точки; Аренду офиса; Коммунальные услуги; 
Аренда складов.

1.20. Покупка наличности
Описание: Комиссии за снятие наличных денег в банкоматах или банках для нужд компании. 
Группа: Выбытие.

1.21. Прочие операционные расходы
Описание: Все мелкие расходы, не вошедшие в другие статьи. Эта статья выбирается если 
остальные не подходят.
Группа: Выбытие.
Включает: Незначительные закупки до 10 000; Одноразовые платежи третьим лицам.

2. Денежные потоки от инвестиционной деятельности
2.1. Покупка основных средств (ОС)
Описание: Приобретение дорогостоящего имущества для ведение бизнеса с долгой амортизацией.
Группа: Выбытие.
Включает: Оборудование; Мебель; Технику; Торгового оборудования; Складского оборудования.

2.2. Ремонт основных средств
Описание: Капитальный ремонт, увеличивающий срок службы дорогостоящего имущества.
Группа: Выбытие.
Включает: Замену частей оборудования; Реставрацию объектов.

2.3. Продажа основных средств
Описание: Доходы от реализации дорогостоящего имущества.
Группа: Поступление.
Включает: Продажу оборудования; Продажу мебели; Продажу техники.

2.4. Выдача кредитов и займов
Описание: Деньги которые мы предоставили третьим лицам - сотрудникам, поставщикам, 
партнерам.
Группа: Выбытие.
Включает: Кредиты контрагентам; Займы сотрудникам; Одолженные сотрудникам.

2.5. Возврат кредитов и займов
Описание: Поступления от погашения долгов. Деньги которые нам вернули третьи лица - 
сотрудники, сотрудницы, поставщики, партнеры.
Группа: Поступление.
Включает: Возврат основной суммы; Проценты по займам.

3. Денежные потоки от финансовой деятельности
3.1. Получение кредитов и займов
Описание: Привлеченные заемные средства. Деньги которые предоставили и предоставляют 
нам банки, поставщики, партнеры.
Группа: Поступление.
Включает: Банковские кредиты; Коммерческие займы.

3.2. Оплаты по кредитам и займам
Описание: Погашение долговых обязательств. Деньги которые мы вернули и возвращаем 
банкам, поставщикам, партнерам.
Группа: Выбытие.
Включает: Основной долг; Проценты по кредитам.

3.3. Вклады от собственников
Описание: Деньги, вложенные учредителями в бизнес. 
Группа: Поступление.
Включает: Дополнительный капитал; Иные взносы.

3.4. Дивиденды
Описание: Выплаты собственникам. 
Группа: Выбытие.
Включает: Часть прибыли; Другие дистрибутивные выплаты.

3.5. Прочие поступления от финансовых операций
Описание: Доходы от финансовых активов. Проценты или дивиденды полученные от вложения 
в другие компании.
Группа: Поступление.
Включает: Проценты на остаток; Дивиденды от акций; Купонные доходы.

Технические операции
4.1. Перевод между счетами
Описание: Перевод денег на другой наш счет или кошелёк.
 \n\n
'''

model_for_accounting = 'gpt-4o-mini-2024-07-18'

temperature_for_accounting = 0.1

In [22]:
# функция для извлечения важных сущностей из сообщении и заполнения таблицы
def accounting(system, model, temperature, db, note, df, verbose=1):

    docs = db.max_marginal_relevance_search(note, k=3, fetch_k=12, lambda_mult=0.5)
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'\n===========Document №{i+1}\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]
    ))

    # if verbose:
    #     print(f' chunks: {message_content}')

    user_for_accounting = f'''
Давай действовать последовательно:

Шаг 1: Извлекай дату операции из сообщения. Если дата указана — добавляй её в
список в формате ДД.ММ.ГГГГ. Если дата в сообщении отсутствует — ставь «-».

Шаг 2: Извлекай сумму средств из сообщения. Если сумма указана — добавляй её в
список в виде положительного числа. Если сумма в сообщении отсутствует — ставь
«-».

Шаг 3: Извлекай счёт из сообщения. Если счёт указан — вноси в список последние
четыре цифры счёта. Если счёт в сообщении отсутствует — ставь «-».

Шаг 4: Извлекай только имя организации или название организации  контрагента.
Если имя организации или название организации контрагента указано в сообщении —
вноси его в список без ковычек. Если имя контрагента или название организации
контрагента отсутствует — ставь «-».

Шаг 5: Определи финансовую статью на основании предоставленной тебе документации.
Статьи находятся во втором уровне документов. Если удаётся определить статью —
точно внеси её полное название(оно стоит после цифр) точно так, как указано в
документации. Не добавляй ничего кроме названия статьи. Если определить статью
невозможно — ставь «-».

Шаг 6. Выведи полученный список из 5 сущностей(элементов списка). \n\n

Сообщение: {note} \n\n {message_content} \n\n
Ответ:
'''

    assistant = "['25.08.2025', 52000, 3123, 'Мелница','Продажи через торговые точки']"

    messages = [
        {'role':'system','content':system},
        {'role':'user','content':user_for_accounting},
        {'role':'assistant', 'content':assistant}
        ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n accounting: \n', answer)

    # обработчик ошибок
    try:
        result = ast.literal_eval(answer)
        if not isinstance(result, list):
            result = ['Error - it is not list']
    except:
        result = ['Error - accounting can not convert']

    # значения флага по умолчанию
    was_written_to_sheet = False

    # условие по определению полноты данных, если все данные получены происходит
    # запись данных в таблицу
    if result.count('-') == 0:
        #
        note_to_sheet(result, note, df)
        #
        was_written_to_sheet = True

    return result, was_written_to_sheet

accointing

## ask

In [23]:
# определяем роль для модели уточнения, версию и температуру
system_for_ask = '''
Ты прекрасный сотрудник в финансовом отделе торговой компании. У тебя прекрасно
получается определять нужно ли задавать уточняющие вопросы, а так же ты умеешь
их задавать. \n\n

Твоя основная задача - проследить чтобы все поля в таблице были заполнены. Тебе
будет предоставлятся список сущностей: [дата, сумма, счет, контрагент, описание].
Ознакомся с ним и там где «-», где отсутсутсвуют данные, задай вопросы для
уточнения. \n\n

Пожалуйста, не сообщай, что заполняешь список.
'''

model_for_ask = 'gpt-4o-mini-2024-07-18'

temperature_for_ask = 0

In [24]:
# функция для уточнения недостоющих данных
def ask(system, model, temperature, note, verbose=1):

    user_for_ask = f'''
Пожалуйста, ознакомся со списком {note}. \n\n

Давай действовать последовательно: \n
Шаг 1: Есть ли в списке «-», иначе сразу закончи диалог.
Шаг 2: Определяем недостающие сущности. \n
Шаг 3: Формируем вопросы для уточнения. \n
Шаг 4: Выводим только лишь вопрос(-ы) для уточнения. \n\n

Пожалуйста, если требуется задай все вопрос(-ы) в одном сообщении.
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_ask}
    ]



    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n ask: \n', answer)

    return answer

### assistant для тест ask

In [25]:
#
system_for_assistant = '''
Ты прекрасный помошник для тестирования. Твоя основная придумывать ответы на
вопросы которые к тебе приходят. \n\n

Пожалуйста, будь краткой и отвечай только по сути вопроса.
Не приветствуй и не проси задавать дополнительных вопросов.

'''


model_for_assistant = 'gpt-4o-mini-2024-07-18'

temperature_for_assistant = 0

In [ ]:
#
def assistant(system, model, temperature, questions, summary, verbose=0):

    user_for_assistant = f'''
Пожалуйста, давай действовать последовательно:
Шаг 1. Ознакомся с контекстом диалога: {summary}
Шаг 2. Ознакомся с вопросами: {questions}
Шаг 2. Если тебя просят уточнить дату, то придумай любую дату в 2025 году, день 
и месяц произвольные. Если не просят уточнить дату, то пропускаешь ответ.
Шаг 3. Если тебя просят уточнить сумму, то придумай любую сумму. Если не просят 
уточнить сумму, то пропускаешь ответ.
Шаг 4. Если тебя просят уточнить счет, то назови счет *3452. Если не просят 
уточнить счет, то пропускаешь ответ.
Шаг 5. Если тебя просят уточнить контрагента, то придумай любую компанию. Если 
не просят уточнить контрагента, то пропускаешь ответ.
Шаг 6. Если тебя просят дать описание, то придумай любое описание операции. 
Если тебя не просят дать описание, то пропускаешь ответ.
Шаг 7. Выведи ответы из предыдущих шагов. \n\n

Ответы:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_assistant}
    ]

    completions = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completions.choices[0].message.content

    if verbose:
        print('\n assistant: \n', answer)

    return answer

## memory

In [27]:
# определяем роль модели, версию и температуру
system_for_memory = '''
Ты замечательный саммаризатор, у тебя отлично получается излагать краткое
содержание текстов. Тебе необходимо саммаризировать текста для подачи в
информации в другие модели ChatGPT. \n\n
'''

model_for_memory = 'gpt-4o-mini-2024-07-18'

temperature_for_memory = 0

In [28]:
# функция для сохранения контекста диалога
def memory(system, model, temperature, text, verbose=1):

    user_for_memory = f'''
Пожалуйста, саммаризируй текст: {text}
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_memory}
    ]



    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n memory: \n', answer)

    return answer

## analyze

In [29]:
# прописываем роль, версию, температура модели
system_for_analyze = '''
Тебя зовут Астрид. Ты великолепный финансист в торговой компании, у тебя
изумительно получается анализировать отчеты. Пожалуйста, будь внимательным
и точным в цифрах.

Тебе предоставлен отчет компании и тебе нужно сформулировать аналитическую записку
для руководства компании. \n\n

Ты знаешь что отчеты могут быть: \n
 - ОДДС или Отчет о движении денежных средств \n
 - ББ или Бухгалтерский баланс \n
 - ОПиУ или Отчет о прибыли и убытках.
'''

model_for_analyze = 'gpt-4o-mini-2024-07-18'

temperature_for_analyze = 0

In [30]:
# функция для анализа отчетности
def analyze(system, model, temperature, sheet, verbose=1):

    user_for_analyze = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Определи вид предоставленного отчета. \n
Шаг 2: Проанализируй отчет учитывая данные из Шаг 1. \n
Шаг 3: Напиши аналитическую записку для руководства. \n\n

Отвечай, пожалуйста, точно, и ничего не придумывай от себя.\n\n

Предоставленный тебе отчет: {sheet}
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_analyze}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n analyze: \n', answer)

    return answer

## router

In [31]:
# определяем роль, версию и температуру модели для аналитики
system_for_router = '''
Ты — умный маршрутизатор, который определяет, к какой модели нужно обратиться
для корректного ответа. Ты работаешь вместе с нейро-финансистом, задача
которого — вести журнал операций и готовить аналитические записки. \n\n

Ты выбираешь одну из трех моделей: 'accounting', 'analyze', 'error'.
\n\n

Выбирай модель по следующим правилам:

- Если сообщение — это ответ (описание операции) → выбери 'accounting'.
- Если сообщение — это аналитический запрос → выбери 'analyze'.

Важно: твой ответ должен быть **только одной строкой**, содержащей имя модели.
'''


model_for_router = 'gpt-4o-mini-2024-07-18'
# model_for_router = 'gpt-4.1-mini-2025-04-14'

temperature_for_router = 0

In [32]:
# функция маршрутизатор
def router(system, model, temperature, note, summary, verbose=1):

    user_for_router = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Ознакомся с контекстом диалога. \n
Шаг 2: Проанализируй сообщение сотрудника. \n
Шаг 3: Определи тип сообщения: запрос, ответ, полный список,
неполный список. \n
Шаг 4: На основе Шаг 1 и Шаг 3 напиши одну модель для ответа сотрудникам. \n\n

Сообщение клиента: {note} \n\n
Контекст диалога: {summary} \n\n
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_router}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n router: \n', answer)

    return answer

# нейро-сотрудник

In [33]:
# готовый нейро-сотрудник после объединения всех агентов и функции
def run_dialog(text, total_summary=''):
    # выводим полученное сообщение
    print('request: ', text)

    if text == 'stop':
        out_answer = '\n end'
        return print(out_answer)
    else:
        note = text
        total_summary += 'Сотрудник: '
        total_summary += note

        # задействуем маршрутизатор
        output = router(system_for_router, model_for_router, temperature_for_router,
                        text, total_summary)

        # условие по задействованию агентов
        if 'accounting' in output:

            out_answer, was_written = accounting(system_for_accounting,
                                                    model_for_accounting,
                                                    temperature_for_accounting,
                                                    db, total_summary, df_total)

            # проверка значения флага очистки контекста после получения всех данных
            if was_written == True:
                total_summary = ''
                out_answer = 'stop'
            else:
                questions = ask(system_for_ask, model_for_ask, temperature_for_ask,
                                out_answer)
                # вызов ассистента для уточнения информации и отправка ответа в роутер
                out_answer = assistant(system_for_assistant, model_for_assistant,
                                        temperature_for_assistant, questions, total_summary)
                summary = out_answer
                total_summary += 'Сотрудник: '
                total_summary += summary

                return run_dialog(out_answer, total_summary)

        elif 'analyze' in output:

            out_answer = analyze(system_for_analyze, model_for_analyze,
                                    temperature_for_analyze, text)

            # вывод ответа агента - аналитика
            return out_answer
        
        elif 'error' in output:
            
            out_answer = 'what?'
            return out_answer

        else:
            return print('Error router')

# тесты

## тесты accounting

In [34]:
# функция для тестирования агента на массиве данных
def test_accounting(df, df_total):

    # перебираем в цикле заметки из массива
    for i in range(df.shape[0]):
        # извлекаем заметку и отправляем в агент accounting
        out = df.loc[i, 'note']
        result = run_dialog(out)

        print('timeout')
        time.sleep(3)

    return df_total

In [35]:
test_accounting(df_ok, df_total)

request:  12.03.24 в магазине на Ленина сегодня выручка была 78 с копейками, оплатили в основном картой, наличка меньше. Деньги пришли на счёт *2487, всё закрыл.

 router: 
 accounting

 accounting: 
 ['12.03.2024', 78, '2487', '-', 'Продажи через торговые точки']

 ask: 
 Какой контрагент был задействован в данной операции?

 assistant: 
 Контрагент: ООО "Торговая сеть".
request:  Контрагент: ООО "Торговая сеть".

 router: 
 accounting

 accounting: 
 ['12.03.2024', 78, 2487, 'ООО Торговая сеть', 'Продажи через торговые точки']
timeout
request:  25 ноября в магазине "Экспресс" собрали 52 тысячи, почти всё наличкой, пару раз по QR платили. Деньги пришли на *2313.

 router: 
 accounting

 accounting: 
 ['25.11.2023', 52000, 2313, 'Экспресс', 'Продажи через торговые точки']
timeout
request:  5 июня 2025 выручка в "Модный дом" — около 104 тысяч. Оплаты и картами, и наличкой. Деньги зачислены на счёт *1231.

 router: 
 accounting

 accounting: 
 ['05.06.2025', 104000, 1231, 'Модный дом', '

,note,date of oper,sum,account,counterparty,category,date of note,id
0,Сотрудник: 12.03.24 в магазине на Ленина сегод...,12.03.2024,78,2487,ООО Торговая сеть,Продажи через торговые точки,NaN,NaN
1,"Сотрудник: 25 ноября в магазине ""Экспресс"" соб...",25.11.2023,52000,2313,Экспресс,Продажи через торговые точки,NaN,NaN
2,"Сотрудник: 5 июня 2025 выручка в ""Модный дом"" ...",05.06.2025,104000,1231,Модный дом,Продажи через торговые точки,NaN,NaN
3,"Сотрудник: 18 июля 2025 с автоматов в ""Сити-Пл...",18.07.2025,15200,2487,Сити-Плаза,Продажи через вендинговые автоматы,NaN,NaN
4,"Сотрудник: 3 февраля 2025 с автомата в ""Городк...",03.02.2025,33000,2313,Компания А,Продажи через вендинговые автоматы,NaN,NaN
...,...,...,...,...,...,...,...,...
93,Сотрудник: 3 ноября 2023 пришли 80 000 с нашег...,03.11.2023,80000,2313,ООО Примерная Компания,Продажи через торговые точки,NaN,NaN
94,Сотрудник: 7 июня 2025 перевели себе 100 000 с...,07.06.2025,100000,1231,Компания А,Перевод между счетами,NaN,NaN
95,Сотрудник: 7 июня 2025 перевёл 100 000 со *123...,07.06.2025,100000,1231,ООО Финансовые Решения,Прочие операционные расходы,NaN,NaN
96,Сотрудник: 12.03.24 в магазине на Ленина сегод...,12.03.2024,78,2487,ООО Торговая сеть,Продажи через торговые точки,NaN,NaN


In [36]:
# сохранение полученных результатов
df_total.to_csv('test_accounting.csv', index=False)

## тесты ask

In [37]:
# функция для тестирования ask на основе масива
def test_ask(df, df_total):
    # цикл для извлечения заметок
    for i in range(df.shape[0]):
        out = df.loc[i, 'note']
        result = run_dialog(out)

        print('timeout')
        time.sleep(3)

    return df_total

In [38]:
test_ask(df_ask, df_total)

request:  Всем привет! Закрыли смену, выручка за сегодня — 45к. Продали кучу всего: джинсы, свитера, обувь. Отчет и чеки в файлике, кому надо — забирайте.

 router: 
 accounting

 accounting: 
 ['-', 45000, '-', '-', 'Продажи через торговые точки']

 ask: 
 1. Какова дата операции?
2. На каком счете была проведена операция?
3. Кто является контрагентом?

 assistant: 
 1. 15 марта 2025 года.
2. Счет *3452.
3. Компания "Модный стиль".
request:  1. 15 марта 2025 года.
2. Счет *3452.
3. Компания "Модный стиль".

 router: 
 accounting

 accounting: 
 ['15.03.2025', 45000, 3452, 'Модный стиль', 'Продажи через торговые точки']
timeout
request:  Только что продали классную кожаную куртку за 12.5к (размер 46, черная). Кто еще хочет такую же — пока есть в наличии! 🚀 Чек прикрепила.

 router: 
 accounting

 accounting: 
 ['-', 12500, '-', '-', 'Продажи через торговые точки']

 ask: 
 1. Какова дата данной операции?
2. На какой счет была зачислена сумма?
3. Кто является контрагентом в данной сделк

,note,date of oper,sum,account,counterparty,category,date of note,id
0,Сотрудник: 12.03.24 в магазине на Ленина сегод...,12.03.2024,78,2487,ООО Торговая сеть,Продажи через торговые точки,NaN,NaN
1,"Сотрудник: 25 ноября в магазине ""Экспресс"" соб...",25.11.2023,52000,2313,Экспресс,Продажи через торговые точки,NaN,NaN
2,"Сотрудник: 5 июня 2025 выручка в ""Модный дом"" ...",05.06.2025,104000,1231,Модный дом,Продажи через торговые точки,NaN,NaN
3,"Сотрудник: 18 июля 2025 с автоматов в ""Сити-Пл...",18.07.2025,15200,2487,Сити-Плаза,Продажи через вендинговые автоматы,NaN,NaN
4,"Сотрудник: 3 февраля 2025 с автомата в ""Городк...",03.02.2025,33000,2313,Компания А,Продажи через вендинговые автоматы,NaN,NaN
...,...,...,...,...,...,...,...,...
193,Сотрудник: Зачислен купонный доход по ранее пр...,15.03.2025,8500,3452,ООО Финансовые решения,Прочие поступления от финансовых операций,NaN,NaN
194,Сотрудник: Перевела 200 000 руб. с основного р...,12.08.2025,200000,3452,Финансовые решения,Перевод между счетами,NaN,NaN
195,Сотрудник: Осуществила перевод 150 000 руб. с ...,15.03.2025,150000,3452,Финансовые решения,Перевод между счетами,NaN,NaN
196,Сотрудник: Перечислила 50 000 руб. с текущего ...,12.08.2025,30000,3452,Финансовые Услуги,Прочие операционные расходы,NaN,NaN


In [39]:
df_total.to_csv('test_ask.csv', index=False)

## тесты analyze

In [40]:
# тестирование агента - analyze
analyze(system_for_analyze, model_for_analyze, temperature_for_analyze, df_sheet)


 analyze: 
 ### Шаг 1: Определение вида предоставленного отчета

Предоставленный отчет является Отчетом о движении денежных средств (ОДДС), так как он содержит информацию о денежных потоках от операционной, инвестиционной и финансовой деятельности, а также о начале и конце месяца.

### Шаг 2: Анализ отчета

1. **Операционная деятельность**:
   - Общие денежные потоки от операционной деятельности составили (605,000.00) рублей.
   - Основные источники поступлений:
     - Продажи через торговые точки: 1,200,000.00 рублей
     - Продажи через вендинговые автоматы: 350,000.00 рублей
     - Возвраты от поставщиков: 80,000.00 рублей
   - Основные расходы:
     - Закупка товара: (900,000.00) рублей
     - Зарплаты и налоги (ФОТ): (525,000.00) рублей (включая производственные, коммерческие и административные расходы)
     - Аренда: (180,000.00) рублей
   - Итог: Операционная деятельность показала отрицательный денежный поток, что может указывать на высокие операционные расходы.

2. **Инвестици

'### Шаг 1: Определение вида предоставленного отчета\n\nПредоставленный отчет является Отчетом о движении денежных средств (ОДДС), так как он содержит информацию о денежных потоках от операционной, инвестиционной и финансовой деятельности, а также о начале и конце месяца.\n\n### Шаг 2: Анализ отчета\n\n1. **Операционная деятельность**:\n   - Общие денежные потоки от операционной деятельности составили (605,000.00) рублей.\n   - Основные источники поступлений:\n     - Продажи через торговые точки: 1,200,000.00 рублей\n     - Продажи через вендинговые автоматы: 350,000.00 рублей\n     - Возвраты от поставщиков: 80,000.00 рублей\n   - Основные расходы:\n     - Закупка товара: (900,000.00) рублей\n     - Зарплаты и налоги (ФОТ): (525,000.00) рублей (включая производственные, коммерческие и административные расходы)\n     - Аренда: (180,000.00) рублей\n   - Итог: Операционная деятельность показала отрицательный денежный поток, что может указывать на высокие операционные расходы.\n\n2. **Инв

# тест сотрудника